In [2]:
import csv
import pandas as pd 

In [3]:
# transform data format from txt to tsv files
with open('phenotype.csv', 'w+',newline='') as csvfile:
    spamwriter = csv.writer(csvfile, dialect='excel')
    with open('phenotype.txt', 'r',encoding='utf-8') as filein:
        for line in filein:
            line_list = line.strip('\n').split('\t')
            spamwriter.writerow(line_list)

In [4]:
# Control delimiters, rows, column names with read_csv
data = pd.read_csv("phenotype.csv", encoding= 'unicode_escape') [['DatabaseID', 'DiseaseName', 'HPO_ID']]
# Preview the first 5 lines of the loaded data
data.head()

C:\Users\dongx\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,DatabaseID,DiseaseName,HPO_ID
0,OMIM:210100,"BETA-AMINOISOBUTYRIC ACID, URINARY EXCRETION OF",HP:0000007
1,OMIM:210100,"BETA-AMINOISOBUTYRIC ACID, URINARY EXCRETION OF",HP:0003355
2,OMIM:163600,NIPPLES INVERTED,HP:0000006
3,OMIM:163600,NIPPLES INVERTED,HP:0003186
4,OMIM:615763,"#615763 CORTICAL DYSPLASIA, COMPLEX, WITH OTHE...",HP:0002365


In [5]:
for item in data['DatabaseID']:
    count = 0
    if 'OMIM' not in item and 'DECIPHER' not in item and 'ORPHA' not in item:
        count += 1
print('# of no record for ID: ' + str(count))

# of no record for ID: 0


In [6]:
for idx in range(len(data['DiseaseName'])):
    data.iloc[idx]['DiseaseName'] = data.iloc[idx]['DiseaseName'].strip()
    item = data.iloc[idx]['DiseaseName']
    # continue if length smaller than 7
    if len(item) < 7:
        continue
    # get rid of data with duplicate OMIM information in Disease Name
    count = 0
    for i in range(6):
        if item[i] in '0123456789':
            count += 1
    if count == 6:
        data.iloc[idx]['DiseaseName'] = item[7:]
        continue
    # get rid of data with duplicate OMIM information and special characters in Disease Name
    for num in '0123456789':
        if (item[0] == '#' or item[0] == '%' or item[0] == '*' or item[0] == '+') and num in item:
            data.iloc[idx]['DiseaseName'] = item[8:]
            continue

In [7]:
count = 0
for item in data['DiseaseName']:
    for num in '0123456789':
        if (item[0] == '#' or item[0] == '%' or item[0] == '*' or item[0] == '+') and num in item:
            count += 1
print('# of wrong format for DiseaseName: ' + str(count))

# of wrong format for DiseaseName: 0


In [8]:
data

,DatabaseID,DiseaseName,HPO_ID
0,OMIM:210100,"BETA-AMINOISOBUTYRIC ACID, URINARY EXCRETION OF",HP:0000007
1,OMIM:210100,"BETA-AMINOISOBUTYRIC ACID, URINARY EXCRETION OF",HP:0003355
2,OMIM:163600,NIPPLES INVERTED,HP:0000006
3,OMIM:163600,NIPPLES INVERTED,HP:0003186
4,OMIM:615763,"CORTICAL DYSPLASIA, COMPLEX, WITH OTHER BRAIN ...",HP:0002365
...,...,...,...
195202,ORPHA:1777,Temtamy syndrome,HP:0002970
195203,ORPHA:1777,Temtamy syndrome,HP:0004209
195204,ORPHA:1777,Temtamy syndrome,HP:0005692
195205,ORPHA:1777,Temtamy syndrome,HP:0007370


In [9]:
for item in data['HPO_ID']:
    if len(item) != 10:
        print(item)

In [10]:
'''
def search_info(data, word):
    word = word.upper()
    res_ID_res = []
    HPO_ID_res = []
    for idx in range(len(data['DiseaseName'])):
        item = data.iloc[idx]['DiseaseName']
        if word in item:
            res_ID_res.append(data.iloc[idx]['DatabaseID'])
            HPO_ID_res.append(data.iloc[idx]['HPO_ID'])
    return res_ID_res, HPO_ID_res
print(search_info(data, input()))           
# NIPPLES INVERTED
'''

"\ndef search_info(data, word):\n    word = word.upper()\n    res_ID_res = []\n    HPO_ID_res = []\n    for idx in range(len(data['DiseaseName'])):\n        item = data.iloc[idx]['DiseaseName']\n        if word in item:\n            res_ID_res.append(data.iloc[idx]['DatabaseID'])\n            HPO_ID_res.append(data.iloc[idx]['HPO_ID'])\n    return res_ID_res, HPO_ID_res\nprint(search_info(data, input()))           \n# NIPPLES INVERTED\n"

In [11]:
# data.to_csv('phenotype_database.csv', sep='\t', encoding='utf-8', index=False)

In [12]:
# with open('HPO_to_genes.txt', newline = '') as data_HPO:                                                                                          
#     data_HPO = csv.reader(data_HPO, delimiter='\t')

In [13]:
data_HPO = pd.read_csv('HPO_to_genes.txt', sep="\t")[['HPO_ID', 'HPO-Name']].drop_duplicates(subset=None, keep='first', inplace=False).reset_index()
data = pd.merge(data, data_HPO, how = 'left', left_on='HPO_ID', right_on='HPO_ID')

In [14]:
data_HPO

,index,HPO_ID,HPO-Name
0,0,HP:0001459,1-3 toe syndactyly
1,1,HP:0006088,1-5 finger complete cutaneous syndactyly
2,2,HP:0010708,1-5 finger syndactyly
3,4,HP:0010713,1-5 toe syndactyly
4,5,HP:0030927,1-minute APGAR score of 0
...,...,...,...
8932,567406,HP:0030500,Yellow/white lesions of the macula
8933,567418,HP:0030506,Yellow/white lesions of the retina
8934,567458,HP:0011462,Young adult onset
8935,567466,HP:0002044,Zollinger-Ellison syndrome


In [15]:
data =  data[['DiseaseName', 'DatabaseID', 'HPO_ID', 'HPO-Name']].sort_values(by='DiseaseName').reset_index()

In [18]:
data = data[['DiseaseName', 'DatabaseID', 'HPO_ID', 'HPO-Name']]
data

,DiseaseName,DatabaseID,HPO_ID,HPO-Name
0,10q22.3q23.3 microdeletion syndrome,ORPHA:276413,HP:0100783,Breast aplasia
1,10q22.3q23.3 microdeletion syndrome,ORPHA:276413,HP:0200008,Intestinal polyposis
2,10q22.3q23.3 microdeletion syndrome,ORPHA:276413,HP:0000252,Microcephaly
3,10q22.3q23.3 microdeletion syndrome,ORPHA:276413,HP:0000256,Macrocephaly
4,10q22.3q23.3 microdeletion syndrome,ORPHA:276413,HP:0000286,Epicanthus
...,...,...,...,...
195202,Åland Islands eye disease,ORPHA:178333,HP:0007663,Reduced visual acuity
195203,Åland Islands eye disease,ORPHA:178333,HP:0007750,Hypoplasia of the fovea
195204,Åland Islands eye disease,ORPHA:178333,HP:0007894,Hypopigmentation of the fundus
195205,Åland Islands eye disease,ORPHA:178333,HP:0001419,X-linked recessive inheritance


In [19]:
data.to_csv('phenotype_database.csv', sep='\t', encoding='utf-8', index=False)